<a href="https://colab.research.google.com/github/nandhu1585/NLP/blob/main/Assignment_9nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


In [ ]:
data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

# Split data into English and French sentences
english_texts, french_texts = zip(*data)
english_texts, french_texts = list(english_texts), list(french_texts)


In [ ]:
# Define tokenizers
english_tokenizer = Tokenizer()
french_tokenizer = Tokenizer()

# Fit the tokenizers on the English and French texts
english_tokenizer.fit_on_texts(english_texts)
french_tokenizer.fit_on_texts(french_texts)

# Convert text to sequences of integers
english_sequences = english_tokenizer.texts_to_sequences(english_texts)
french_sequences = french_tokenizer.texts_to_sequences(french_texts)

# Pad sequences to ensure uniform length
max_english_length = max(len(seq) for seq in english_sequences)
max_french_length = max(len(seq) for seq in french_sequences)
english_sequences = pad_sequences(english_sequences, maxlen=max_english_length, padding='post')
french_sequences = pad_sequences(french_sequences, maxlen=max_french_length, padding='post')

# Get the vocabulary sizes
english_vocab_size = len(english_tokenizer.word_index) + 1
french_vocab_size = len(french_tokenizer.word_index) + 1


In [ ]:
# Create decoder input and target sequences
decoder_input_data = np.array([seq[:-1] for seq in french_sequences])
decoder_target_data = np.array([seq[1:] for seq in french_sequences])


In [ ]:
# Define model parameters
latent_dim = 256  # Dimensionality of the LSTM layer

# Encoder
encoder_inputs = Input(shape=(max_english_length,))
encoder_embedding = tf.keras.layers.Embedding(english_vocab_size, latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_french_length-1,))
decoder_embedding = tf.keras.layers.Embedding(french_vocab_size, latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(french_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 4)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 4, 256)         │          3,840 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 2, 256)         │          3,584 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 256), (None,   │        525,312 │ embedding[0][0]        │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 2, 256),       │        525,312 │ embedding_1[0][0],     │
│                           │ (None, 256), (None,    │                │ lstm[0][1], lstm[0][2] │
│                           │ 256)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 2, 14)          │          3,598 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,061,646 (4.05 MB)

 Trainable params: 1,061,646 (4.05 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Expand dimensions to match model requirements
decoder_target_data = np.expand_dims(decoder_target_data, -1)

# Train the model
model.fit([english_sequences, decoder_input_data], decoder_target_data, batch_size=64, epochs=100)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0714 - loss: 2.6378
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.4286 - loss: 2.5637
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.4286 - loss: 2.4916
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.4286 - loss: 2.4006
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.4286 - loss: 2.2758
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.4286 - loss: 2.1053
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.4286 - loss: 1.9034
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.4286 - loss: 1.7417
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.4286 - loss: 1.6595
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.4286 - loss: 1.6064
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.4286 - loss: 1.5598
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.5714 - 

In [ ]:
# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model for inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [ ]:
def translate_sentence(input_text):
    # Tokenize input text
    input_sequence = english_tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=max_english_length, padding='post')

    # Encode the input as state vectors
    states_value = encoder_model.predict(input_sequence)

    # Generate empty target sequence of length 1
    target_sequence = np.zeros((1, 1))
    # The first token in the target sequence is the start token (assume index 1)
    target_sequence[0, 0] = french_tokenizer.word_index.get("start", 1)

    # Sampling loop for a batch of sequences
    stop_condition = False
    translated_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_sequence] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = french_tokenizer.index_word.get(sampled_token_index, "?")

        # Exit condition: either hit max length or find stop token
        if sampled_word == "end" or len(translated_sentence.split()) > max_french_length:
            stop_condition = True
        else:
            translated_sentence += " " + sampled_word

        # Update the target sequence (of length 1)
        target_sequence = np.zeros((1, 1))
        target_sequence[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return translated_sentence.strip()


In [ ]:
new_sentence = "hello"
translated = translate_sentence(new_sentence)
print(f"English: {new_sentence}")
print(f"French: {translated}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
English: hello
French: ? ? ? ?
